In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz

C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
# products_sheet_id = "1FB2bbvWE2nsf49KO_DkWzbL2hm9r4G8j"
# products_list = "Tablib%20Dataset"
# url = f"https://docs.google.com/spreadsheets/d/{products_sheet_id}/gviz/tq?tqx=out:csv&sheet={products_list}"

In [2]:
# convert csv datasets into dataframes
pdt_df = pd.read_csv('full_products_list.csv')
batches_df = pd.read_csv('batches_list.csv')
recipe_df = pd.read_csv('recipe_list.csv')

In [3]:
# import excel file from an absolute path with pandas

xls = pd.ExcelFile(r'C:\Users\NDA\Desktop\EVE\TMCG\documents\duplicate_products.xlsx')

In [4]:
# convert xls datasets on sheet names into dataframes

dp_list = pd.read_excel(xls, 'copy_final_list_duplicates')
unique_pdts_df = pd.read_excel(xls, 'unique final list')

In [5]:
# drop unwanted columns from the duplicates list dataframe

new_dp_list = dp_list.drop(['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 8',
                           'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14',
                           'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
                           'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23'], axis=1)

In [6]:
# cleanup data by renaming pandas series with more meaningfull names

new_dp_list = new_dp_list.rename(columns={'Eva': 'PRODUCT ID', 'Unnamed: 6': 'GENERIC NAME', 'Unnamed: 7': 'BRAND NAME'})

In [7]:
# drop null values based on rows

new_dp_list = new_dp_list.dropna(axis='rows')

In [8]:
# remove the inverse labels

new_dp_list = new_dp_list.drop([new_dp_list.index[-1]])

In [9]:
# convert product id into numeric 

new_dp_list['PRODUCT ID'] = pd.to_numeric(new_dp_list['PRODUCT ID'], errors='ignore')

In [10]:
# drop unnecessary columns from unique lists

unique_pdts_df = unique_pdts_df.drop(['COUNTRY OF ORIGIN', 'FORMULATION', 'RETAIL PRICE PER SMALLEST UNIT', 
                                      'RETAIL PRICE PER SMALLEST UNIT.1', 'PACK SIZE', 'STRENGTH', 'eshop_id', 
                                      'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 
                                      'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 
                                      'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 
                                      'Unnamed: 25'], axis=1)

In [11]:
# change column name into product id for easy manipulation

unique_pdts_df = unique_pdts_df.rename(columns={'ID': 'PRODUCT ID'})

In [12]:
# give proper indices to the dataset

new_dp_list.reset_index(drop=True, inplace=True)

In [13]:
unique_dict = {k:[] for k in new_dp_list['PRODUCT ID']}

for i in range(len(unique_pdts_df)):
    
   # loop through the list of duplicates using its index
    for j in range(len(new_dp_list)):
        
       # compare the two pandas series on generic names. Coerce them to lowercase for proper comparison
        if unique_pdts_df['GENERIC NAME'][i].lower() in new_dp_list['GENERIC NAME'][j].lower():
            
            unique_dict[new_dp_list['PRODUCT ID'][j]].append(unique_pdts_df['PRODUCT ID'][i])
            
unique_dict

{3951: [3352],
 387: [715],
 4933: [715],
 4942: [715],
 3146: [],
 5312: [830],
 5416: [],
 3819: [],
 5304: [5303],
 5239: [],
 4685: [4686],
 4672: [],
 5444: [569],
 4896: [166],
 4888: [166],
 209: [166],
 5033: [5533],
 530: [],
 418: [],
 261: [],
 4477: [5134],
 4865: [4855],
 3915: [638],
 3904: [638],
 3304: [638],
 253: [],
 5234: [],
 3325: [3925],
 2725: [3925],
 3952: [3352, 4594],
 4595: [715],
 719: [696],
 4057: [4056],
 4132: [3532],
 5391: [],
 5390: [],
 5389: [],
 5031: [902],
 394: [902],
 902: [902],
 4031: [4029],
 5378: [5347],
 4729: [715],
 5370: [647],
 540: [],
 4868: [4867],
 5380: [],
 5379: [],
 3814: [2614],
 538: [173],
 636: [5152],
 5394: [4442]}

In [15]:
# loop through the unique products dataframe using its index
for i in range(len(unique_pdts_df)):
    
    # loop through the list of duplicates using its index
    for j in range(len(new_dp_list)):
        if len(unique_dict[new_dp_list['PRODUCT ID'][j]]) == 0:
        
            # extract string from the product
            str_a  = unique_pdts_df['GENERIC NAME'][i].lower()
            str_b = new_dp_list['GENERIC NAME'][j].lower()

            # assign ratio to both strings
            ratio = fuzz.partial_ratio(str_b, str_a)            

            # sent conditon for comparing the series
            if ratio >= 81:
                unique_dict[new_dp_list['PRODUCT ID'][j]].append(unique_pdts_df['PRODUCT ID'][i])
unique_dict

{3951: [3352],
 387: [715],
 4933: [715],
 4942: [715],
 3146: [],
 5312: [830],
 5416: [4745],
 3819: [3816],
 5304: [5303],
 5239: [5044],
 4685: [4686],
 4672: [],
 5444: [569],
 4896: [166],
 4888: [166],
 209: [166],
 5033: [5533],
 530: [4786],
 418: [4786],
 261: [4786],
 4477: [5134],
 4865: [4855],
 3915: [638],
 3904: [638],
 3304: [638],
 253: [4786],
 5234: [5208],
 3325: [3925],
 2725: [3925],
 3952: [3352, 4594],
 4595: [715],
 719: [696],
 4057: [4056],
 4132: [3532],
 5391: [],
 5390: [],
 5389: [],
 5031: [902],
 394: [902],
 902: [902],
 4031: [4029],
 5378: [5347],
 4729: [715],
 5370: [647],
 540: [],
 4868: [4867],
 5380: [5337],
 5379: [5337],
 3814: [2614],
 538: [173],
 636: [5152],
 5394: [4442]}

In [16]:
unique_dict[540].append(346)
unique_dict[5391].append(4334)
unique_dict[5389].append(4334)
unique_dict[5390].append(4334)
unique_dict[4672].append(5145)
unique_dict[3146].append(4328)
unique_dict[3819].append(5303)
unique_dict[4729].append(4945)
unique_dict[4595].append(4594)
unique_dict[3819].pop(0)
unique_dict[4595].pop(0)
unique_dict[4729].pop(0)
unique_dict[3952].pop(1)

4594

### The product dataset

In [101]:
# change product's id for ease in comparing with other dataframes
new_pdt_df = pdt_df.rename(columns={'ID': 'PRODUCT ID'})

In [102]:
# check to find out whether the new_product_df id is identical to the duplicate list id 
merged_pdt_df = new_pdt_df['PRODUCT ID'].isin(new_dp_list['PRODUCT ID'])

In [103]:
# drop the identical ids
new_pdt_df.drop(new_pdt_df[merged_pdt_df].index, inplace=True)

In [104]:
# checkout the new dataframe
new_pdt_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2554 entries, 0 to 2603
Data columns (total 10 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   PRODUCT ID                        2554 non-null   int64  
 1   COUNTRY OF ORIGIN                 1218 non-null   object 
 2   FORMULATION                       1484 non-null   object 
 3   RETAIL PRICE PER SMALLEST UNIT    2532 non-null   float64
 4   RETAIL PRICE PER SMALLEST UNIT.1  2483 non-null   float64
 5   PACK SIZE                         2554 non-null   int64  
 6   GENERIC NAME                      2554 non-null   object 
 7   BRAND NAME                        2554 non-null   object 
 8   STRENGTH                          2229 non-null   object 
 9   eshop_id                          389 non-null    float64
dtypes: float64(3), int64(2), object(5)
memory usage: 219.5+ KB


In [105]:
# test on a product from the duplicates list
new_pdt_df[new_pdt_df['PRODUCT ID'] == 387]

,PRODUCT ID,COUNTRY OF ORIGIN,FORMULATION,RETAIL PRICE PER SMALLEST UNIT,RETAIL PRICE PER SMALLEST UNIT.1,PACK SIZE,GENERIC NAME,BRAND NAME,STRENGTH,eshop_id


In [ ]:
# merged_pdt_df[merged_pdt_df['PRODUCT ID'] == 3951]

### The batches dataframe

In [ ]:
batches_df.head(20)

In [ ]:
batches_df[batches_df['PRODUCT ID'] == 3352]

In [17]:
unique_dict

{3951: [3352],
 387: [715],
 4933: [715],
 4942: [715],
 3146: [4328],
 5312: [830],
 5416: [4745],
 3819: [5303],
 5304: [5303],
 5239: [5044],
 4685: [4686],
 4672: [5145],
 5444: [569],
 4896: [166],
 4888: [166],
 209: [166],
 5033: [5533],
 530: [4786],
 418: [4786],
 261: [4786],
 4477: [5134],
 4865: [4855],
 3915: [638],
 3904: [638],
 3304: [638],
 253: [4786],
 5234: [5208],
 3325: [3925],
 2725: [3925],
 3952: [3352],
 4595: [4594],
 719: [696],
 4057: [4056],
 4132: [3532],
 5391: [4334],
 5390: [4334],
 5389: [4334],
 5031: [902],
 394: [902],
 902: [902],
 4031: [4029],
 5378: [5347],
 4729: [4945],
 5370: [647],
 540: [346],
 4868: [4867],
 5380: [5337],
 5379: [5337],
 3814: [2614],
 538: [173],
 636: [5152],
 5394: [4442]}

In [18]:
for key, val in unique_dict.items():
    if key in batches_df['PRODUCT ID']:
        batches_df['NEW PRODUCT ID'] = batches_df['PRODUCT ID'].map(unique_dict, na_action='ignore')
        batches_df['NEW PRODUCT ID'] = batches_df['NEW PRODUCT ID'].str.get(0)
print(batches_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3982 entries, 0 to 3981
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   PRODUCT ID             3982 non-null   int64  
 1   id                     3982 non-null   int64  
 2   created                3982 non-null   object 
 3   modified               3982 non-null   object 
 4   batch_number           3982 non-null   object 
 5   cost_price             3982 non-null   int64  
 6   initial_item_count     3982 non-null   int64  
 7   item_count             3982 non-null   int64  
 8   expiration_date        3982 non-null   object 
 9   manufacturing_date     1179 non-null   object 
 10  product__generic_name  3982 non-null   object 
 11  product__brand_name    3982 non-null   object 
 12  NEW PRODUCT ID         71 non-null     float64
dtypes: float64(1), int64(5), object(7)
memory usage: 404.5+ KB
None


In [90]:
batches_df['NEW PRODUCT ID'] = batches_df['NEW PRODUCT ID'].fillna(0)

In [92]:
batches_df['NEW PRODUCT ID'] = batches_df['NEW PRODUCT ID'].astype(int)

In [95]:
for i in range(len(batches_df)):
    if batches_df['NEW PRODUCT ID'][i] != 0:
#         print(batches_df.head())
        batches_df['PRODUCT ID'][i] = batches_df['NEW PRODUCT ID'][i]

<ipython-input-95-3dc723979feb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batches_df['PRODUCT ID'][i] = batches_df['NEW PRODUCT ID'][i]


In [99]:
batches_df[batches_df['PRODUCT ID'] == 5347]

,PRODUCT ID,id,created,modified,batch_number,cost_price,initial_item_count,item_count,expiration_date,manufacturing_date,product__generic_name,product__brand_name,NEW PRODUCT ID
1698,5347,2378,2021-01-05 18:21:41,2021-01-05 18:21:41,CE-2262-A,610,0,0,2022-01-31,NaN,FLUCONAZOLE,FLUCAZ 100MG,5347
1699,5347,2377,2021-01-05 18:20:37,2021-01-05 18:20:40,CE-2262-A,610,0,0,2021-01-31,NaN,FLUCONAZOLE,FLUCAZ 100MG,5347


In [93]:
batches_df.head(20)

,PRODUCT ID,id,created,modified,batch_number,cost_price,initial_item_count,item_count,expiration_date,manufacturing_date,product__generic_name,product__brand_name,NEW PRODUCT ID
0,5567,4087,2021-05-30 20:49:51,2021-05-30 20:49:51,9104KYU,1500,120,0,2022-05-31,NaN,Nifedipine 30mg,ADALAT,0
1,848,4086,2021-05-30 20:49:17,2021-05-30 20:49:17,77371DR,800,60,0,2022-05-31,NaN,Metoprolol,METOPROLOL 50MG UK,0
2,5637,4085,2021-05-30 20:48:50,2021-05-30 20:48:50,BBN,2400,120,0,2022-05-31,NaN,Tacrolimus,TACROLIMUS PROGRAF 1MG,0
3,5663,4084,2021-05-30 20:48:21,2021-05-30 20:48:21,WWC,3500,90,0,2022-05-31,NaN,Mycophenolate Mofetil,CELLCEPT 500MG TAB,0
4,5485,4083,2021-05-30 20:21:11,2021-05-30 20:21:11,9104BN,2100,30,0,2022-05-31,NaN,"Sitagliptin, Metformin",JANUMET 50/500 MG,0
5,247,4082,2021-05-30 18:38:59,2021-05-30 18:38:59,W0002D,11500,5,0,2023-01-30,NaN,Xylometazoline,XYLO ACINO 0.1%,0
6,389,4081,2021-05-30 17:19:35,2021-05-30 17:19:35,2007403,18500,3,0,2023-09-30,NaN,Ceftriaxone,EPICEPHIN 2G INJ,0
7,4925,4080,2021-05-30 17:08:06,2021-05-30 17:08:06,T026363,60000,2,0,2022-07-30,NaN,Mometasone Nasal Spray 120 Sprays,Nasonex,0
8,4786,4079,2021-05-30 14:08:29,2021-05-30 14:08:29,A0236002,750,70,0,2022-12-02,NaN,Loratidine,MOSEDINE,4786
9,286,4078,2021-05-30 09:03:54,2021-05-30 09:03:54,Y106XK,124,100,0,2022-08-30,NaN,"Paracetamol,Caffeine",PANADOL EXTRA,0


### The recipe dataset

In [106]:
recipe_df.columns

Index(['id', 'created', 'modified', 'prescription', 'product_id', 'product',
       'dosage', 'tablets', 'quantity', 'frequency', 'duration', 'route',
       'price_discount', 'serving_pharmacy', 'price_controller', 'rx_price',
       'external_price', 'requires_approval', 'approved', 'dropped',
       'canceling_party', 'reason_for_cancelation', 'authorized_by_tmcg',
       'authorized_by_insurer', 'client_decision', 'insurer_code'],
      dtype='object')

In [108]:
for key, val in unique_dict.items():
    if key in recipe_df['product_id']:
        recipe_df['NEW PRODUCT ID'] = recipe_df['product_id'].map(unique_dict, na_action='ignore')
        recipe_df['NEW PRODUCT ID'] = recipe_df['NEW PRODUCT ID'].str.get(0)
print(batches_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3982 entries, 0 to 3981
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   PRODUCT ID             3982 non-null   int64 
 1   id                     3982 non-null   int64 
 2   created                3982 non-null   object
 3   modified               3982 non-null   object
 4   batch_number           3982 non-null   object
 5   cost_price             3982 non-null   int64 
 6   initial_item_count     3982 non-null   int64 
 7   item_count             3982 non-null   int64 
 8   expiration_date        3982 non-null   object
 9   manufacturing_date     1179 non-null   object
 10  product__generic_name  3982 non-null   object
 11  product__brand_name    3982 non-null   object
 12  NEW PRODUCT ID         3982 non-null   int32 
dtypes: int32(1), int64(5), object(7)
memory usage: 389.0+ KB
None


In [110]:
recipe_df.head()

,id,created,modified,prescription,product_id,product,dosage,tablets,quantity,frequency,...,requires_approval,approved,dropped,canceling_party,reason_for_cancelation,authorized_by_tmcg,authorized_by_insurer,client_decision,insurer_code,NEW PRODUCT ID
0,20077,2021-05-31 12:06:21,2021-05-31 12:06:21,11008,807,VALUPACK VITAMIN C (Vitamin C),1,1.0,1,o.d,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20076,2021-05-31 11:59:36,2021-05-31 11:59:36,11007,642,LEVOBACT 750MG (Levofloxacin),750mg,1.0,5,o.d,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20075,2021-05-31 11:59:36,2021-05-31 11:59:36,11007,334,FOLIC ACID (Generic) (Folic Acid),1tab,1.0,14,o.d,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20074,2021-05-31 11:59:36,2021-05-31 11:59:36,11007,537,LUCIN CREAM 15G (Hydrocortisone),1,1.0,1,o.d,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20073,2021-05-31 11:59:36,2021-05-31 11:59:36,11007,225,PANTOP-D (Pantoprazole/Domperidone),1tab,1.0,30,o.d,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [114]:
recipe_df[recipe_df['product_id'] == 253]

,id,created,modified,prescription,product_id,product,dosage,tablets,quantity,frequency,...,requires_approval,approved,dropped,canceling_party,reason_for_cancelation,authorized_by_tmcg,authorized_by_insurer,client_decision,insurer_code,NEW PRODUCT ID
13,20064,2021-05-31 11:50:54,2021-05-31 11:50:54,11004,253,AERIUS 5MG (Desloratidine),1 tab,1.0,7,o.d,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,4786
194,19882,2021-05-30 12:21:12,2021-05-30 20:43:45,10932,253,AERIUS (Desloratidine 5mg),5mg,1.0,5,o.d,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,4786
245,19829,2021-05-30 09:48:32,2021-05-30 09:48:32,10913,253,AERIUS 5MG (Desloratidine),5 mg,1.0,5,o.d,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,4786
261,19813,2021-05-30 09:04:09,2021-05-30 09:04:09,10903,253,AERIUS 5MG (Desloratidine),5 mg,1.0,5,o.d,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,4786
290,19781,2021-05-29 17:59:04,2021-05-29 17:59:04,10891,253,AERIUS 5MG (Desloratidine),1 tab,1.0,5,o.d,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,4786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14768,5192,2020-11-07 15:05:44,2020-11-07 15:05:44,3213,253,Desloratidine (AERIUS),5 mg,1.0,7,o.d,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,4786
14814,5146,2020-11-06 16:26:34,2020-11-06 16:26:34,3187,253,Desloratidine (AERIUS),5 mg,1.0,7,o.d,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,4786
14837,5123,2020-11-06 12:21:07,2020-11-06 12:21:07,3176,253,Desloratidine (AERIUS),5 mgs,1.0,7,o.d,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,4786
14842,5117,2020-11-06 11:27:40,2020-11-06 11:27:40,3173,253,Desloratidine (AERIUS),5 mgs,1.0,7,o.d,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,4786


In [112]:
recipe_df['NEW PRODUCT ID'] = recipe_df['NEW PRODUCT ID'].fillna(0)

In [113]:
recipe_df['NEW PRODUCT ID'] = recipe_df['NEW PRODUCT ID'].astype(int)

In [116]:
for i in range(len(recipe_df)):
    if recipe_df['NEW PRODUCT ID'][i] != 0:
#         print(batches_df.head())
        recipe_df['product_id'][i] = recipe_df['NEW PRODUCT ID'][i]

<ipython-input-116-13c03043f44b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recipe_df['product_id'][i] = recipe_df['NEW PRODUCT ID'][i]


In [117]:
recipe_df

,id,created,modified,prescription,product_id,product,dosage,tablets,quantity,frequency,...,requires_approval,approved,dropped,canceling_party,reason_for_cancelation,authorized_by_tmcg,authorized_by_insurer,client_decision,insurer_code,NEW PRODUCT ID
0,20077,2021-05-31 12:06:21,2021-05-31 12:06:21,11008,807,VALUPACK VITAMIN C (Vitamin C),1,1.0,1,o.d,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0
1,20076,2021-05-31 11:59:36,2021-05-31 11:59:36,11007,642,LEVOBACT 750MG (Levofloxacin),750mg,1.0,5,o.d,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0
2,20075,2021-05-31 11:59:36,2021-05-31 11:59:36,11007,334,FOLIC ACID (Generic) (Folic Acid),1tab,1.0,14,o.d,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0
3,20074,2021-05-31 11:59:36,2021-05-31 11:59:36,11007,537,LUCIN CREAM 15G (Hydrocortisone),1,1.0,1,o.d,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0
4,20073,2021-05-31 11:59:36,2021-05-31 11:59:36,11007,225,PANTOP-D (Pantoprazole/Domperidone),1tab,1.0,30,o.d,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19825,47,2019-08-22 16:19:30,2019-08-22 16:19:30,50,155,Amoxicillin/Clavulanic(CLAVULIN 625),400mg,1.0,1,t.d.s,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0
19826,46,2019-08-22 16:19:30,2019-08-22 16:19:30,50,167,Metronidazole(FLAGYL),400mg,1.0,15,t.d.s,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0
19827,45,2019-08-16 08:00:51,2020-06-14 16:12:43,49,674,Levofloxacin ( LEVOXIN),500mg,1.0,5,o.d,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0
19828,38,2019-08-11 12:54:17,2019-08-11 12:54:17,44,135,Amoxicillin(DURAMOX),500,1.0,1,b.d,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0


In [121]:
recipe_df[recipe_df['product_id'] == 394]

,id,created,modified,prescription,product_id,product,dosage,tablets,quantity,frequency,...,requires_approval,approved,dropped,canceling_party,reason_for_cancelation,authorized_by_tmcg,authorized_by_insurer,client_decision,insurer_code,NEW PRODUCT ID


In [122]:
new_products_data = new_pdt_df.to_csv(r'C:\Users\NDA\Desktop\EVE\TMCG\documents\new_products_data.csv')

In [124]:
new_batches_data = batches_df.drop('NEW PRODUCT ID', axis=1)

In [125]:
new_batches_data.to_csv(r'C:\Users\NDA\Desktop\EVE\TMCG\documents\new_batches_data.csv')

In [126]:
new_recipe_data = recipe_df.drop('NEW PRODUCT ID', axis=1)

In [127]:
new_recipe_data.to_csv(r'C:\Users\NDA\Desktop\EVE\TMCG\documents\new_recipe_data.csv')